In [ ]:
import os
import json
import requests
from openai import OpenAI
from typing import Dict, List
from datetime import datetime, timedelta

from langchain.schema import Document
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

from transformers import pipeline

In [ ]:
def fetch_news_articles(source: str, query: str, days_back: int = 30) -> List[Document]:
    
    from_date = (datetime.now() - timedelta(days=days_back)).strftime('%Y-%m-%d')
    url = f"https://newsapi.org/v2/everything?q={query}&from={from_date}&apiKey={NEWSAPI_KEY}"
    
    try:
        response = requests.get(url)
        response.raise_for_status()
        articles = response.json().get("articles", [])
        
        documents = []
        for article in articles:
            content = article.get("content") or article.get("description") or ""
            if content:
                metadata = {
                    "source": article.get("url", "Unknown"),
                    "title": article.get("title", "No title"),
                    "publishedAt": article.get("publishedAt", "Unknown")
                }
                documents.append(Document(page_content=content, metadata=metadata))
        
        return documents
    except requests.RequestException as e:
        print(f"Error fetching articles from {source}: {e}")
        return []

In [4]:
def load_vectorstore(persist_dir: str, source: str, query: str) -> Chroma:

    embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

    if os.path.exists(persist_dir):
        print(f"Loading existing vectorstore from: {persist_dir}")
        return Chroma(persist_directory=persist_dir, embedding_function=embedding_model)

    print(f"Creating new vectorstore at: {persist_dir}")
    os.makedirs(persist_dir, exist_ok=True)

    documents = fetch_news_articles(source, query)
    if not documents:
        print(f"No articles found for {source}. Using empty vectorstore.")
        return Chroma(embedding_function=embedding_model, persist_directory=persist_dir)

    splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = splitter.split_documents(documents)

    vectorstore = Chroma.from_documents(chunks, embedding=embedding_model, persist_directory=persist_dir)
    vectorstore.persist()

    print(f"Vectorstore created and saved at: {persist_dir}")
    return vectorstore

In [ ]:
NASA_QUERY = "climate change nasa"
NCEI_QUERY = "climate change noaa"
PERSIST_DIR_NASA = "./archive/chroma_store_nasa"
PERSIST_DIR_NCEI = "./archive/chroma_store_ncei"

support_vectorstore = load_vectorstore(persist_dir=PERSIST_DIR_NASA, source="nasa.gov", query=NASA_QUERY)
oppose_vectorstore = load_vectorstore(persist_dir=PERSIST_DIR_NCEI, source="noaa.gov", query=NCEI_QUERY)

C:\Users\nhatt\AppData\Local\Temp\ipykernel_11824\823364932.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



Loading existing vectorstore from: chroma_store_nasa


C:\Users\nhatt\AppData\Local\Temp\ipykernel_11824\823364932.py:7: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  return Chroma(persist_directory=persist_dir, embedding_function=embedding_model)


Loading existing vectorstore from: chroma_store_ncei


In [7]:
sentiment_analyzer = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=1)
intent_analyzer = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

def analyze_sentiment(statement: str) -> Dict:
    """Phân tích cảm xúc của tuyên bố."""
    try:
        result = sentiment_analyzer(statement)
        if isinstance(result, list) and len(result) > 0 and isinstance(result[0], list) and len(result[0]) > 0 and "label" in result[0][0]:
            return {"emotion": result[0][0]["label"].lower(), "score": result[0][0]["score"]}
        else:
            return {"emotion": "unknown", "score": 0.0, "error": "Invalid result format"}
    except Exception as e:
        return {"emotion": "unknown", "score": 0.0, "error": f"Sentiment analysis failed: {str(e)}"}

def analyze_intent(statement: str) -> Dict:
    """Phân tích ý định của tuyên bố."""
    candidate_labels = ["informative", "persuasive", "controversial", "misleading", "alarmist"]
    try:
        result = intent_analyzer(statement, candidate_labels, multi_label=False)
        if isinstance(result, dict) and "labels" in result and "scores" in result and len(result["labels"]) > 0:
            return {"intent": result["labels"][0].lower(), "score": result["scores"][0]}
        else:
            return {"intent": "unknown", "score": 0.0, "error": "Invalid result format"}
    except Exception as e:
        return {"intent": "unknown", "score": 0.0, "error": f"Intent analysis failed: {str(e)}"}

Device set to use cpu
Device set to use cpu


In [ ]:
client = OpenAI(api_key="your_openai")

def clean_response(response: str) -> str:
    response = response.strip()
    if response.startswith("```") or response.startswith("'''"):
        response = response.strip("`").strip("'").strip()
        if response.lower().startswith("json"):
            response = response[4:].strip()
    return response.strip()


def query_Model(prompt: str, max_retries: int = 3) -> dict:

    full_prompt = prompt.strip() + "\nReturn the response strictly in JSON format with keys 'Final Verdict' and 'Explanation'. No Markdown, no formatting, no extra text."

    for attempt in range(max_retries):
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "user", "content": full_prompt}
            ],
            temperature=0.2,
            top_p=0.9,
            max_tokens=1024,
        )

        raw_text = response.choices[0].message.content.strip()
        return clean_response(raw_text)


In [23]:
verdictsClimateFeedback = ["Credible", "Not Credible"]

In [24]:
statement_task = f'''
 You are a fact-checking AI assistant specializing in scientific information.
# Your task is to analyze statements and categorize their accuracy using the following categories: {verdictsClimateFeedback}. Each level describes common characteristics and examples to guide your analysis:
---
## Credible
Assign this label if the statement:
- Is supported by strong scientific evidence or aligns with the established scientific consensus.
- May lack some nuance or context, but remains fundamentally accurate and not misleading.
**Examples**:
- *"Babies under six months should not drink water as it can result in health risks."*  
  → **Credible**. Backed by medical guidelines on infant hydration.
- *"Prioritizing plant-based foods reduces greenhouse gas emissions."*  
  → **Credible**. Generally supported by environmental research, though effects depend on scale of adoption.
- *"Climate change will destroy all ecosystems."*  
  → **Credible**, if interpreted with added context. While some ecosystems may adapt, many are at severe risk.
---
## Not Credible
Assign this label if the statement:
- Contradicts scientific consensus or lacks reliable evidence.
- Is based on flawed reasoning, misleading framing, or speculative claims without scientific backing.
- Contains factual inaccuracies that distort public understanding of the issue.
**Examples**:
- *"CO2 increases are mainly due to natural causes, not humans."*  
  → **Not Credible**. Overwhelming evidence shows fossil fuel emissions are the primary cause.
- *"Greenland's ice cores show no significant warming, disproving climate change."*  
  → **Not Credible**. Scientific data confirms Greenland’s warming trends.
- *"The Atlantic is cooling, and scientists don't know why."*  
  → **Not Credible**. Misleading due to cherry-picking short-term anomalies.
---
## Instructions
1. **Analyze** the statement's factual accuracy and scientific reasoning.
2. **Choose one label**: `Credible` or `Not Credible`.
3. **Provide a brief explanation** justifying the label.
4. **If necessary**, identify missing context or evidence and explain why it matters.
'''

In [35]:
def claimant_assess(statement: str, vectorstore=None, role: str = "Support", follow_up_questions: List[str] = None, other_claiman_context: str = None) -> Dict:
    sentiment_result = analyze_sentiment(statement)
    intent_result = analyze_intent(statement)

    context = ""
    if vectorstore:
        docs = vectorstore.similarity_search(statement, k=3)
        context_lines = [f"{doc.page_content} (Source: {doc.metadata.get('source', 'Unknown')}, Page: {doc.metadata.get('page', 'N/A')}, URL: {doc.metadata.get('url', 'N/A')})" for doc in docs]
        context = "\n".join(context_lines)

    context += f"\nSentiment: {sentiment_result['emotion']} (Confidence: {sentiment_result['score']:.2f})\nIntent: {intent_result['intent']} (Confidence: {intent_result['score']:.2f})"

    # Listing 2: Advocate Primer
    advocate_primer = f'''
    You are an AI Advocate, responsible for fact-checking user statements about climate change.
    Your expertise draws on a specific set of trustworthy scientific documents, and your responses must be grounded in the evidence provided.
    ### Expertise
    You specialize in:
    - Climate change
    - Climate science
    - Environmental science
    - Physics
    - Energy science
    - Science communication
    ### Objective
    Your main objective is to verify the accuracy of user statements on climate change by examining the evidence in your assigned documents.
    ### Crucial Instructions
    * **Evidence is Essential**: Your responses *must* be directly supported by the information in your assigned documents.
    * **Cite Sources**: Always reference 'Reference', 'Page', and 'URL' when citing evidence.
    * **Avoid Speculation**: If there is not enough information to assess a statement, state "Not Enough Information" and explain what specific information is missing.
    * **Follow StaetmentTask Instructions**: Use the verdict categories provided in StatementTask: {statement_task}, and ensure your evaluation aligns with those instructions.
    * **Consider Sentiment and Intent**: Note if the statement's sentiment or intent suggests a potential to spread misinformation.
    ### Assessment Process
    1. **Evaluate the Statement**
    - Analyze based on expertise and evidence in assigned documents.
    - Use verdict categories from StatementTask.
    - Determine if evidence supports, contradicts, or lacks information.
    - Consider sentiment and intent for potential misinformation impact.
    2. **Optional: Break Down into Substatements**
    - Divide complex statements into smaller substatements.
    - Evaluate each substatement individually.
    3. **Synthesize an Overall Verdict**
    - Consider accuracy, importance, and sentiment/intent.
    - Assign an overall verdict based on synthesis.
    **Output Format**:
    {{
        "Final Verdict": "",
        "Explanation": ""
    }}
    If not enough information:
    {{
        "Final Verdict": "Not Enough Information",
        "Explanation": "I do not have enough information to assess this statement. [Explain missing-information.]"
    }}
    '''

    # Listing 3: GPT-4 Primer
    gpt4_primer = f'''
    You are an AI Statementant, responsible for fact-checking user statements about climate change.
    Your expertise is based on general scientific knowledge, and your responses must reflect the consensus in climate science.
    ### Objective
    Verify the accuracy of user statements on climate change.
    ### Crucial Instructions
    * **Avoid Speculation**: If there is not enough information, state "Not Enough Information" and explain what is missing.
    * **Follow StatementTask Instructions**: Use verdict categories from StatementTask: {statement_task}.
    * **Consider Sentiment and Intent**: Note if the statement's sentiment or intent suggests misinformation potential.
    ### Assessment Process
    1. **Evaluate the Statement**
    - Analyze based on scientific consensus.
    - Use verdict categories from ClaimTask.
    - Consider sentiment and intent.
    2. **Optional: Break Down into Substatements**
    - Divide complex statements and evaluate individually.
    3. **Synthesize an Overall Verdict**
    - Assign based on accuracy and sentiment/intent.
    **Output Format**:
    {{
        "Final Verdict": "",
        "Explanation": ""
    }}
    If not enough information:
    {{
        "Final Verdict": "Not Enough Information",
        "Explanation": "I do not have enough information to assess this statement. [Explain missing-information.]"
    }}
    '''

    # Listing 5: Denier Advocate Primer
    denier_primer = f'''
    You are an AI Denier Advocate, critically evaluating user statements about climate change from a skeptical perspective.
    Your expertise is strictly limited to the specific set of documents provided.
    ### Objective
    Scrutinize the accuracy of user statements based solely on assigned documents.
    ### Crucial Instructions
    * **Use Only Provided Documents**: Responses must be based exclusively on document content.
    * **Strict Evidence Requirement**: Respond with "Not Enough Information" if documents lack sufficient data.
    * **Cite Sources Precisely**: Provide 'Reference', 'Page', and 'URL'.
    * **Avoid External Knowledge**: Do not use general scientific principles unless in documents.
    * **Follow StatementTask Instructions**: Use verdict categories from StatementTask: {statement_task}.
    * **Consider Sentiment and Intent**: Note misinformation potential.
    ### Assessment Process
    1. **Evaluate Based on Documents**
    - Assess strictly within document scope.
    - Use StatementTask verdict categories.
    - Consider sentiment and intent.
    2. **Optional: Break Down into Substatements**
    - Separate and evaluate individually.
    3. **Final Verdict for Simple Statements**
    - Assess as a whole if straightforward.
    **Output Format**:
    {{
        "Final Verdict": "",
        "Explanation": ""
    }}
    If not enough information:
    {{
        "Final Verdict": "Not Enough Information",
        "Explanation": "The documents provided do not contain enough information to assess this statement. [Explain missing-information.]"
    }}
    '''

    primer = {
        "Support": advocate_primer,
        "Denier": denier_primer,
        "GPT4": gpt4_primer
    }.get(role, advocate_primer)

    follow_up_instruction = f"Address these questions: {follow_up_questions}" if follow_up_questions else ""
    other_claimant_instruction = f"Consider other Claimants: {other_claiman_context}" if other_claiman_context else ""

    prompt = f"""
    {primer}
    Statement: '{statement}'
    Data: {context}
    {follow_up_instruction}
    {other_claimant_instruction}
    Return the response strictly in JSON format with keys 'Final Verdict' (string, not list) and 'Explanation'. No Markdown or other formatting.
    """
    response = query_Model(prompt)
    result = json.loads(response)

    return {
        "role": role,
        "Final Verdict": result.get("Final Verdict", "Not Enough Information"),
        "Explanation": f"{result.get('Explanation', response)}\n(Sentiment: {sentiment_result['emotion']}, Intent: {intent_result['intent']})",
        "Sentiment": sentiment_result,
        "Intent": intent_result
    }

In [26]:
def verifier_evaluate(statement: str, claimant_responses: List[Dict], round_number: int = 1, max_rounds: int = 5) -> Dict:
    context = [{"Claimant": r["role"], "Verdict": r["Final Verdict"], "Explanation": r["Explanation"], "Sentiment": r["Sentiment"], "Intent": r["Intent"]} for r in claimant_responses]
    context_str = json.dumps(context, indent=2)

    # Listing 4: Verifier Primer
    verifier_primer = f'''
    Role: Authoritative "Verifier" System
    Primary Objective: Synthesize assessments from Claimants to determine the veracity of a user's statement on climate change, prioritizing scientific rigor.
    ### Expertise
    You specialize in:
    - Climate change
    - Climate science
    - Environmental science
    - Physics
    - Energy science
    - Science communication
    ### Responsibilities
    1. Review verdicts and explanations from Claimants.
    2. Consolidate into a final verdict, prioritizing evidence-based assessments.
    3. Seek clarification through follow-up questions if discrepancies arise.
    4. Prioritize Claimants with specific, high-quality evidence.
    5. Consider sentiment and intent for misinformation potential.
    ### Final Assessment Criteria
    1. Analyze collective Claimant assessments.
    2. Do not rely on majority voting; prioritize evidence quality.
    3. Assess sentiment and intent impact on misinformation.
    ### Guidelines
    1. Prioritize Claimants with concrete evidence over those with "Not Enough Information."
    2. Ask follow-up questions if Claimants provide conflicting evidence or if additional evidence is needed to strengthen the verdict.
    3. Stop debate if no Claimant changes assessment after follow-up or if evidence is sufficient.
    4. Cite 'Reference', 'Page', and 'URL' when referring to Claimant data.
    5. Assess risk level (High, Medium, Low) based on sentiment/intent.
    **Output Format**:
    {{
        "Final Verdict": "",
        "Explanation": "",
        "Confidence": "high|medium|low",
        "RiskLevel": "high|medium|low",
        "Debate": {{
            "Status": "Open|Closed",
            "Round": {round_number},
            "Follow-Up Questions": {{}}
        }}
    }}
    '''

    verdicts = [r["Final Verdict"] for r in claimant_responses]
    valid_verdicts = [v for v in verdicts if v != "Not Enough Information"]
    unique_verdicts = set(valid_verdicts)

    risk_level = "low"
    sentiment_warnings = []
    for r in claimant_responses:
        sentiment = r.get("Sentiment", {"emotion": "unknown", "score": 0.0})
        intent = r.get("Intent", {"intent": "unknown", "score": 0.0})
        if sentiment["emotion"] in ["anger", "fear"] or intent["intent"] in ["misleading", "controversial"]:
            risk_level = "high"
            sentiment_warnings.append(f"Warning: {r['role']} detected {sentiment['emotion']} sentiment or {intent['intent']} intent, increasing risk of misinformation spread.")
        elif sentiment["score"] > 0.5 or intent["score"] > 0.5:
            risk_level = max(risk_level, "medium")

    confidence = "high" if len(unique_verdicts) == 1 and len(valid_verdicts) >= 2 else "medium" if valid_verdicts else "low"
    debate_status = "Open" if len(unique_verdicts) > 1 or (len(valid_verdicts) == 1 and len(valid_verdicts) < len(verdicts) and confidence != "high") else "Closed"

    follow_up_questions = {}
    if debate_status == "Open" and round_number < max_rounds: 
        for r in claimant_responses:
            if r["Final Verdict"] == "Not Enough Information":
                follow_up_questions[r["role"]] = [
                    f"Can you provide specific evidence or data from your resources that directly addresses the statement '{statement}'?",
                    f"What additional information or analysis is needed to evaluate the accuracy of the statement, and why is it missing from your current resources?"
                ]
            elif r["role"] == "Denier":
                follow_up_questions[r["role"]] = [
                    f"Can you clarify the methodologies or data sources in your cited studies that support your assessment of the statement '{statement}'?",
                    f"How do your findings compare to the broader scientific consensus or mainstream climate science regarding this statement?"
                ]
            else:  # Support
                follow_up_questions[r["role"]] = [
                    f"Can you provide detailed evidence or studies from your resources to support or refute the statement '{statement}'?",
                    f"How does your assessment align with the broader scientific consensus or documented evidence on this topic?"
                ]

    prompt = f"""
    {verifier_primer}
    Statement: '{statement}'
    Claimants: {context_str}
    Debate Status: {debate_status}
    Confidence: {confidence}
    Follow-Up Questions: {json.dumps(follow_up_questions, indent=2)}
    """

    response = query_Model(prompt)
    result = json.loads(response)

    final_verdict = result.get("Final Verdict", "Not Enough Information")
    if final_verdict not in verdictsClimateFeedback:
        final_verdict = "Not Enough Information"

    result["Final Verdict"] = final_verdict
    result["Confidence"] = confidence
    result["RiskLevel"] = risk_level
    result["Explanation"] += "\n" + "\n".join(sentiment_warnings) if sentiment_warnings else ""
    result["Debate"] = {
        "Status": debate_status,
        "Round": round_number,
        "Follow-Up Questions": follow_up_questions
    }

    return result

In [27]:
def neutral_verifier_evaluate(statement: str, claimant_responses: List[Dict], verifier_result: Dict, round_number: int = 1, max_rounds: int = 5) -> Dict:
    context = [{"Claimant": r["role"], "Verdict": r["Final Verdict"], "Explanation": r["Explanation"], "Sentiment": r["Sentiment"], "Intent": r["Intent"]} for r in claimant_responses]
    context.append({"Verifier": "ClimateExpert", "Verdict": verifier_result["Final Verdict"], "Explanation": verifier_result["Explanation"]})
    context_str = json.dumps(context, indent=2)

    # Listing 6: Neutral Verifier Primer
    neutral_verifier_primer = f'''
    Role: Neutral "Verifier" System
    Primary Objective: Synthesize assessments from Claimants and Verifier to determine the final veracity of a user's statement, ensuring impartiality.
    ### Responsibilities
    1. Review verdicts and explanations from Claimants and Verifier.
    2. Consolidate into a final verdict, prioritizing evidence-based assessments.
    3. Seek clarification through follow-up questions if discrepancies arise.
    4. Prioritize judgments with specific evidence.
    5. Consider sentiment and intent for misinformation potential.
    ### Final Assessment Criteria
    1. Analyze collective assessments.
    2. Do not rely on majority voting; prioritize evidence quality.
    3. Assess sentiment and intent impact on misinformation.
    ### Guidelines
    1. Prioritize Claimants/Verifier with concrete evidence.
    2. Ask follow-up questions if conflicting evidence exists or if additional evidence is needed.
    3. Stop debate if no changes occur after follow-up or if evidence is sufficient.
    4. Cite 'Reference', 'Page', and 'URL' when referring to data.
    5. Assess risk level (High, Medium, Low) based on sentiment/intent.
    **Output Format**:
    {{
        "Final Verdict": "",
        "Explanation": "",
        "Confidence": "high|medium|low",
        "RiskLevel": "high|medium|low",
        "Debate": {{
            "Status": "Open|Closed",
            "Round": {round_number},
            "Follow-Up Questions": {{}}
        }}
    }}
    '''

    verdicts = [r["Final Verdict"] for r in claimant_responses] + [verifier_result["Final Verdict"]]
    valid_verdicts = [v for v in verdicts if v != "Not Enough Information"]
    unique_verdicts = set(valid_verdicts)

    risk_level = "low"
    sentiment_warnings = []
    for r in claimant_responses:
        sentiment = r.get("Sentiment", {"emotion": "unknown", "score": 0.0})
        intent = r.get("Intent", {"intent": "unknown", "score": 0.0})
        if sentiment["emotion"] in ["anger", "fear"] or intent["intent"] in ["misleading", "controversial"]:
            risk_level = "high"
            sentiment_warnings.append(f"Warning: {r['role']} detected {sentiment['emotion']} sentiment or {intent['intent']} intent, increasing risk of misinformation spread.")
        elif sentiment["score"] > 0.5 or intent["score"] > 0.5:
            risk_level = max(risk_level, "medium")

    confidence = "high" if len(unique_verdicts) == 1 and len(valid_verdicts) >= 2 else "medium" if valid_verdicts else "low"
    debate_status = "Open" if len(unique_verdicts) > 1 or (len(valid_verdicts) == 1 and len(valid_verdicts) < len(verdicts) and confidence != "high") else "Closed"

    follow_up_questions = {}
    if debate_status == "Open" and round_number < max_rounds:  
        for r in claimant_responses:
            if r["Final Verdict"] == "Not Enough Information":
                follow_up_questions[r["role"]] = [
                    f"Can you provide specific evidence or data from your resources that directly addresses the statement '{statement}'?",
                    f"What additional information or analysis is needed to evaluate the accuracy of the statement, and why is it missing from your current resources?"
                ]
            elif r["role"] == "Denier":
                follow_up_questions[r["role"]] = [
                    f"Can you clarify the methodologies or data sources in your cited studies that support your assessment of the statement '{statement}'?",
                    f"How do your findings compare to the broader scientific consensus or mainstream climate science regarding this statement?"
                ]
            else:  # Support 
                follow_up_questions[r["role"]] = [
                    f"Can you provide detailed evidence or studies from your resources to support or refute the statement '{statement}'?",
                    f"How does your assessment align with the broader scientific consensus or documented evidence on this topic?"
                ]
    

    prompt = f"""
    {neutral_verifier_primer}
    Statement: '{statement}'
    Claimants and Verifier: {context_str}
    Debate Status: {debate_status}
    Confidence: {confidence}
    Follow-Up Questions: {json.dumps(follow_up_questions, indent=2)}
    """


    response = query_Model(prompt)
    result = json.loads(response)

    if debate_status == "Open" and round_number >= max_rounds:
        debate_status = "Closed"
        result["Explanation"] += "\nDebate closed due to reaching maximum rounds ({}).".format(max_rounds)

    final_verdict = result.get("Final Verdict", "Not Enough Information")
    if final_verdict not in verdictsClimateFeedback:
        final_verdict = "Not Enough Information"

    result["Final Verdict"] = final_verdict
    result["Confidence"] = confidence
    result["RiskLevel"] = risk_level
    result["Explanation"] += "\n" + "\n".join(sentiment_warnings) if sentiment_warnings else ""
    result["Debate"] = {
        "Status": debate_status,
        "Round": round_number,
        "Follow-Up Questions": follow_up_questions
    }

    return result

real

In [ ]:
def truthseekers(statement: str, max_rounds: int = 5) -> Dict:
    claimants = [
        {"role": "Support", "vectorstore": support_vectorstore},
        {"role": "Denier", "vectorstore": oppose_vectorstore},
        {"role": "GPT4", "vectorstore": None}
    ]

    claimant_responses = []
    round_number = 1

    for claimant in claimants:
        response = claimant_assess(statement, claimant["vectorstore"], claimant["role"])
        print(f"Claimant {claimant['role']} response: {response}")
        claimant_responses.append(response)

    verifier_result = verifier_evaluate(statement, claimant_responses, round_number)
    neutral_verifier_result = neutral_verifier_evaluate(statement, claimant_responses, verifier_result, round_number)
    while neutral_verifier_result["Debate"]["Status"] == "Open" and round_number < max_rounds:
        round_number += 1
        follow_up = neutral_verifier_result["Debate"]["Follow-Up Questions"]
        new_responses = []

        for claimant in claimants:
            role = claimant["role"]
            questions = follow_up.get(role, [])
            if not questions:
                for r in claimant_responses:
                    if r["role"] == role:
                        new_responses.append(r)
                        break
                continue

            others = [f"{r['role']} Claimant: Verdict: {r['Final Verdict']}, Explanation: {r['Explanation']}" for r in claimant_responses if r["role"] != role]
            other_context = "\n".join(others)

            response = claimant_assess(
                statement,
                claimant["vectorstore"],
                role,
                follow_up_questions=questions,
                other_claiman_context=other_context
            )
            new_responses.append(response)

        claimant_responses = new_responses
        verifier_result = verifier_evaluate(statement, claimant_responses, round_number)
        neutral_verifier_result = neutral_verifier_evaluate(statement, claimant_responses, verifier_result, round_number)

        new_verdicts = [r["Final Verdict"] for r in claimant_responses]
        if all(v == "Not Enough Information" for v in new_verdicts) and neutral_verifier_result["Final Verdict"] == "Not Enough Information":
            neutral_verifier_result["Debate"]["Status"] = "Closed"
            neutral_verifier_result["Explanation"] += "\nDebate closed due to no new information provided in round {}.".format(round_number)
            break

    return {
        "statement": statement,
        "final_verdict": neutral_verifier_result.get("Final Verdict", "Not Enough Information"),
        "explanation": neutral_verifier_result.get("Explanation", "No reasoning provided"),
        "confidence": neutral_verifier_result.get("Confidence", "low"),
        "risk_level": neutral_verifier_result.get("RiskLevel", "low"),
        "debate_rounds": round_number,
        "claimant_responses": claimant_responses,
        "verifier_result": verifier_result
    }

In [39]:
statement = "Of course the climate is changing. It always has. It always will"
truthseekers_result = truthseekers(statement)
print(json.dumps(truthseekers_result, indent=2, ensure_ascii=False))

Claimant Support response: {'role': 'Support', 'Final Verdict': 'Not Credible', 'Explanation': "The statement 'Of course the climate is changing. It always has. It always will' is misleading because it implies that current climate change is a natural occurrence similar to past changes. While the climate has indeed changed throughout Earth's history due to natural factors, the current changes are primarily driven by human activities, such as the burning of fossil fuels, which is supported by overwhelming scientific evidence. This statement lacks the necessary context to accurately convey the unique and unprecedented nature of current climate change.\n(Sentiment: neutral, Intent: persuasive)", 'Sentiment': {'emotion': 'neutral', 'score': 0.8974751830101013}, 'Intent': {'intent': 'persuasive', 'score': 0.4130096733570099}}
Claimant Denier response: {'role': 'Denier', 'Final Verdict': 'Not Enough Information', 'Explanation': 'The documents provided do not contain enough information to asse